In [ ]:
import os
import numpy as np
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, WhiteKernel
import plotly.graph_objects as go
import pickle

## Load model

In [ ]:
with open('gpr_rbf.pickle', 'rb') as f:
    model = pickle.load(f)
gpr = model['gpr']
scaler = model['scaler']
del model

## Load data from mat and plot

In [ ]:
data = loadmat('orange_straight_eval.mat')

for i in range(len(data['straight_cuts'][0])):
    cut = data['straight_cuts'][0][i]
    time = cut[:,0]
    y = cut[:,8]
    X = cut[:,[1,5]]
    X[:,0] = [-10*x for x in X[:,0]]
    pred_force, std = gpr.predict(X, return_std=True)
    pred_force = scaler.inverse_transform(pred_force)
    std = 2*np.sqrt(std) * scaler.scale_
    fig = go.Figure(data=[
    go.Scatter(x=time, y=pred_force.ravel() + std, fill=None, line=dict(color='black', dash='dash'), showlegend=False),
    go.Scatter(x=time, y=pred_force.ravel() - std, fill='tonexty', line=dict(color='black', dash='dash'), name='&plusmn; 2*\u03C3 range', fillcolor='rgba(0, 0, 0, 0.2)'),
    go.Scatter(x=time, y=y, marker=dict(color='red'), name='measured'),
    go.Scatter(x=time, y=pred_force.ravel(), marker=dict(color='black'), name='predicted'),
    ])
    fig.update_xaxes(title_text='Time after cut start [s]')
    fig.update_yaxes(title_text='Cutting force [N]')
    fig.show()
